In [74]:
print("Hello from my first data project!")


Hello from my first data project!


In [75]:
import requests   # to call the web API
import pandas as pd   # to work with tables (dataframes)
API_KEY = "b654dc40b92bdd512956b2478836f207278e54d27dca9cdf1f20ab728bd7bd16"


In [76]:
# Countries we care about (Baltics + Eastern Europe)
COUNTRIES = {
    "EE": "Estonia",
    "LV": "Latvia",
    "LT": "Lithuania",
    "PL": "Poland",
    "CZ": "Czechia",
    "SK": "Slovakia",
    "HU": "Hungary",
    "RO": "Romania",
    "BG": "Bulgaria",
    "UA": "Ukraine",
}

COUNTRIES


{'EE': 'Estonia',
 'LV': 'Latvia',
 'LT': 'Lithuania',
 'PL': 'Poland',
 'CZ': 'Czechia',
 'SK': 'Slovakia',
 'HU': 'Hungary',
 'RO': 'Romania',
 'BG': 'Bulgaria',
 'UA': 'Ukraine'}

In [77]:
BASE_URL = "https://api.openaq.org/v3"
BASE_URL


'https://api.openaq.org/v3'

In [78]:
def get_locations_for_country(iso_code, limit=100, page=1):
    url = f"{BASE_URL}/locations"
    params = {
        "iso": iso_code,
        "limit": limit,
        "page": page,
    }
    headers = {
        "X-API-Key": API_KEY
    }
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    return response.json()


In [79]:
latvia_data = get_locations_for_country("LV", limit=10, page=1)

type(latvia_data), list(latvia_data.keys())


(dict, ['meta', 'results'])

In [80]:
len(latvia_data["results"])


10

In [81]:
latvia_data["results"][0]


{'id': 5864,
 'name': 'LV0008A',
 'locality': None,
 'timezone': 'Europe/Riga',
 'country': {'id': 52, 'code': 'LV', 'name': 'Latvia'},
 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'},
 'provider': {'id': 187, 'name': 'EEA Latvia'},
 'isMobile': False,
 'isMonitor': True,
 'instruments': [{'id': 2, 'name': 'Government Monitor'}],
 'sensors': [{'id': 23947,
   'name': 'no2 µg/m³',
   'parameter': {'id': 5,
    'name': 'no2',
    'units': 'µg/m³',
    'displayName': 'NO₂ mass'}},
  {'id': 23948,
   'name': 'o3 µg/m³',
   'parameter': {'id': 3,
    'name': 'o3',
    'units': 'µg/m³',
    'displayName': 'O₃ mass'}},
  {'id': 15914,
   'name': 'so2 µg/m³',
   'parameter': {'id': 6,
    'name': 'so2',
    'units': 'µg/m³',
    'displayName': 'SO₂ mass'}}],
 'coordinates': {'latitude': 56.5102029995063, 'longitude': 27.333369},
 'licenses': [{'id': 10,
   'name': 'ODC-BY',
   'attribution': {'name': 'Unknown Governmental Organization', 'url': None},
   'dateFrom': '2017-09-21

In [82]:
def locations_to_dataframe(json_obj, iso_code):
    """
    Take the JSON from OpenAQ and convert 'results' into a flat table.
    One row per sensor (a station can have multiple sensors).
    """
    rows = []
    for loc in json_obj["results"]:
        sensors = loc.get("sensors", [])
        for sensor in sensors:
            rows.append(
                {
                    "location_id": loc["id"],
                    "location_name": loc.get("name"),
                    "country_code": iso_code,
                    "locality": loc.get("locality"),
                    "timezone": loc.get("timezone"),
                    "latitude": loc["coordinates"]["latitude"],
                    "longitude": loc["coordinates"]["longitude"],
                    "sensor_id": sensor["id"],
                    "parameter": sensor["parameter"]["name"],
                    "units": sensor["parameter"]["units"],
                    "is_monitor": loc.get("isMonitor"),
                    "is_mobile": loc.get("isMobile"),
                }
            )
    return pd.DataFrame(rows)


In [83]:
latvia_df = locations_to_dataframe(latvia_data, "LV")
latvia_df.head()


,location_id,location_name,country_code,locality,timezone,latitude,longitude,sensor_id,parameter,units,is_monitor,is_mobile
0,5864,LV0008A,LV,None,Europe/Riga,56.510203,27.333369,23947,no2,µg/m³,True,False
1,5864,LV0008A,LV,None,Europe/Riga,56.510203,27.333369,23948,o3,µg/m³,True,False
2,5864,LV0008A,LV,None,Europe/Riga,56.510203,27.333369,15914,so2,µg/m³,True,False
3,8216,LV0007A,LV,None,Europe/Riga,57.404633,21.591989,23957,no2,µg/m³,True,False
4,8216,LV0007A,LV,None,Europe/Riga,57.404633,21.591989,23960,o3,µg/m³,True,False


In [84]:
all_locations = []

for iso_code in COUNTRIES.keys():
    print(f"Fetching locations for {iso_code}...")
    page = 1
    while True:
        data = get_locations_for_country(iso_code, limit=100, page=page)
        df = locations_to_dataframe(data, iso_code)

        # If no rows in this page, stop for this country
        if df.empty:
            break

        all_locations.append(df)

        # If this page returned fewer than 100 rows, it's the last page
        if len(df) < 100:
            break

        # Otherwise, go to the next page
        page += 1

# Combine everything into one big table
locations_df = pd.concat(all_locations, ignore_index=True)
locations_df.head()


Fetching locations for EE...
Fetching locations for LV...
Fetching locations for LT...
Fetching locations for PL...
Fetching locations for CZ...
Fetching locations for SK...
Fetching locations for HU...
Fetching locations for RO...
Fetching locations for BG...
Fetching locations for UA...


,location_id,location_name,country_code,locality,timezone,latitude,longitude,sensor_id,parameter,units,is_monitor,is_mobile
0,8056,"""Liivalaia""",EE,Tallinn,Europe/Tallinn,59.431114,24.760565,23364,co,µg/m³,True,False
1,8056,"""Liivalaia""",EE,Tallinn,Europe/Tallinn,59.431114,24.760565,23365,no2,µg/m³,True,False
2,8056,"""Liivalaia""",EE,Tallinn,Europe/Tallinn,59.431114,24.760565,23366,o3,µg/m³,True,False
3,8056,"""Liivalaia""",EE,Tallinn,Europe/Tallinn,59.431114,24.760565,23367,pm10,µg/m³,True,False
4,8056,"""Liivalaia""",EE,Tallinn,Europe/Tallinn,59.431114,24.760565,23368,so2,µg/m³,True,False


In [18]:
locations_df.shape


(5448, 12)

In [85]:
locations_df["country_code"].value_counts()


country_code
PL    1471
RO    1344
BG     887
CZ     591
HU     576
SK     333
LT      88
LV      75
EE      58
UA      25
Name: count, dtype: int64

In [86]:
locations_df["parameter"].value_counts()


parameter
pm10                1081
no2                  828
pm25                 721
so2                  633
o3                   577
co                   496
temperature          229
pm1                  220
relativehumidity     117
um003                117
bc                   108
no                    86
pm4                   85
humidity              75
pressure              75
Name: count, dtype: int64

In [87]:
# Filter for PM2.5 sensors only
pm25_locations = locations_df[
    locations_df["parameter"].str.lower() == "pm25"
]

pm25_locations.shape


(721, 12)

In [88]:
baltic_codes = ["EE", "LV", "LT"]

pm25_baltics = pm25_locations[
    pm25_locations["country_code"].isin(baltic_codes)
]

pm25_baltics.shape


(28, 12)

In [89]:
pm25_baltics.head()


,location_id,location_name,country_code,locality,timezone,latitude,longitude,sensor_id,parameter,units,is_monitor,is_mobile
7,8081,"""Saarejärve""",EE,Saare,Europe/Tallinn,58.702778,26.758898,23435,pm25,µg/m³,True,False
13,8083,"""Kohtla-Järve""",EE,Kohtla-Järve,Europe/Tallinn,59.409725,27.278622,23453,pm25,µg/m³,True,False
18,8084,"""Lahemaa""",EE,Vihula,Europe/Tallinn,59.494446,25.930569,8613037,pm25,µg/m³,True,False
29,8087,"""Õismäe""",EE,Tallinn,Europe/Tallinn,59.414169,24.649458,23448,pm25,µg/m³,True,False
35,10624,"""Tartu""",EE,Nõo,Europe/Tallinn,58.370556,26.734722,35347,pm25,µg/m³,True,False


In [90]:
pm25_baltics["country_code"].value_counts()


country_code
LT    10
EE     9
LV     9
Name: count, dtype: int64

In [91]:
from datetime import date


In [92]:
def get_daily_measurements_for_sensor(sensor_id, date_from, date_to, limit=1000, page=1):
    """
    Fetch daily average measurements for ONE sensor between date_from and date_to.
    Dates must be strings like '2023-01-01'.
    """
    url = f"{BASE_URL}/sensors/{sensor_id}/days"
    params = {
        "date_from": date_from,
        "date_to": date_to,
        "limit": limit,
        "page": page,
    }
    headers = {
        "X-API-Key": "b654dc40b92bdd512956b2478836f207278e54d27dca9cdf1f20ab728bd7bd16"
    }
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    return response.json()


In [93]:
def days_to_dataframe(json_obj, sensor_id):
    """
    Convert the 'days' JSON result for one sensor into a DataFrame.
    Each row = one day.
    """
    rows = []
    for item in json_obj.get("results", []):
        period = item.get("period") or {}
        
        dt_from_obj = period.get("datetimeFrom") or {}
        dt_to_obj   = period.get("datetimeTo") or {}

        dt_from_local = dt_from_obj.get("local")
        dt_to_local   = dt_to_obj.get("local")

        date_local = None
        for dt in (dt_from_local, dt_to_local):
            if dt:
                date_local = dt[:10]  # 'YYYY-MM-DD'
                break

        rows.append(
            {
                "sensor_id": sensor_id,
                "parameter": item["parameter"]["name"],
                "units": item["parameter"]["units"],
                "date_local": date_local,
                "avg_value": item["value"],
            }
        )

    return pd.DataFrame(rows)


In [94]:
pm25_baltics["sensor_id"].head()


7       23435
13      23453
18    8613037
29      23448
35      35347
Name: sensor_id, dtype: int64

In [95]:
test_sensor_id = pm25_baltics["sensor_id"].iloc[0]
test_sensor_id


np.int64(23435)

In [96]:
date_from = "2023-01-01"
date_to = "2023-12-31"

test_json = get_daily_measurements_for_sensor(test_sensor_id, date_from, date_to)
type(test_json), list(test_json.keys())


(dict, ['meta', 'results'])

In [97]:
# Filter for pm25 in all countries
pm25_locations = locations_df[
    locations_df["parameter"].str.lower() == "pm25"
]

# Keep only Baltic countries
baltic_codes = ["EE", "LV", "LT"]

pm25_baltics = pm25_locations[
    pm25_locations["country_code"].isin(baltic_codes)
]

pm25_baltics.shape, pm25_baltics["country_code"].value_counts()


((28, 12),
 country_code
 LT    10
 EE     9
 LV     9
 Name: count, dtype: int64)

In [98]:
pm25_baltics.head()


,location_id,location_name,country_code,locality,timezone,latitude,longitude,sensor_id,parameter,units,is_monitor,is_mobile
7,8081,"""Saarejärve""",EE,Saare,Europe/Tallinn,58.702778,26.758898,23435,pm25,µg/m³,True,False
13,8083,"""Kohtla-Järve""",EE,Kohtla-Järve,Europe/Tallinn,59.409725,27.278622,23453,pm25,µg/m³,True,False
18,8084,"""Lahemaa""",EE,Vihula,Europe/Tallinn,59.494446,25.930569,8613037,pm25,µg/m³,True,False
29,8087,"""Õismäe""",EE,Tallinn,Europe/Tallinn,59.414169,24.649458,23448,pm25,µg/m³,True,False
35,10624,"""Tartu""",EE,Nõo,Europe/Tallinn,58.370556,26.734722,35347,pm25,µg/m³,True,False


In [99]:
pm25_baltic_sensor_ids = pm25_baltics["sensor_id"].unique()[:20]
len(pm25_baltic_sensor_ids), pm25_baltic_sensor_ids[:5]


(20, array([  23435,   23453, 8613037,   23448,   35347]))

In [100]:
date_from = "2023-01-01"
date_to = "2023-12-31"


In [101]:
all_days = []

for sid in pm25_baltic_sensor_ids:
    print(f"Fetching daily data for sensor {sid}...")
    page = 1
    while True:
        data = get_daily_measurements_for_sensor(sid, date_from, date_to, limit=1000, page=page)
        df = days_to_dataframe(data, sid)

        if df.empty:
            # No data returned for this page -> stop for this sensor
            break

        all_days.append(df)

        # If less than 1000 rows came back, it's likely the last page
        if len(df) < 1000:
            break

        page += 1

pm25_baltics_daily = pd.concat(all_days, ignore_index=True)
pm25_baltics_daily.head()


Fetching daily data for sensor 23435...
Fetching daily data for sensor 23453...
Fetching daily data for sensor 8613037...
Fetching daily data for sensor 23448...
Fetching daily data for sensor 35347...
Fetching daily data for sensor 8613075...
Fetching daily data for sensor 14152440...
Fetching daily data for sensor 13502144...
Fetching daily data for sensor 14651174...
Fetching daily data for sensor 1606344...
Fetching daily data for sensor 1606348...
Fetching daily data for sensor 1606345...
Fetching daily data for sensor 7774665...
Fetching daily data for sensor 7773036...
Fetching daily data for sensor 8613644...
Fetching daily data for sensor 7776613...
Fetching daily data for sensor 12178120...
Fetching daily data for sensor 14152350...
Fetching daily data for sensor 5102511...
Fetching daily data for sensor 5078414...


,sensor_id,parameter,units,date_local,avg_value
0,1606344,pm25,µg/m³,2023-01-01,8.11
1,1606344,pm25,µg/m³,2023-01-02,5.73
2,1606344,pm25,µg/m³,2023-01-03,2.86
3,1606344,pm25,µg/m³,2023-01-04,4.36
4,1606344,pm25,µg/m³,2023-01-05,6.09


In [102]:
pm25_baltics_daily.shape


(1333, 5)

In [103]:
pm25_baltics_unique = pm25_baltics.drop_duplicates(subset=["sensor_id"])

pm25_baltics_daily_full = pm25_baltics_daily.merge(
    pm25_baltics_unique[
        ["sensor_id", "country_code", "locality", "location_name", "latitude", "longitude"]
    ],
    on="sensor_id",
    how="left"
)

pm25_baltics_daily_full.head()
pm25_baltics_daily_full.shape


(1333, 10)

In [104]:
import os

os.makedirs("../data/raw", exist_ok=True)



In [105]:
output_path = "../data/raw/pm25_baltics_daily_2023_sample.csv"
pm25_baltics_daily_full.to_csv(output_path, index=False)
output_path


'../data/raw/pm25_baltics_daily_2023_sample.csv'

In [106]:
import os

# Create data/raw if it doesn't already exist
os.makedirs("../data/raw", exist_ok=True)


In [107]:
# Countries we care about
COUNTRIES = {
    "EE": "Estonia",
    "LV": "Latvia",
    "LT": "Lithuania",
    "PL": "Poland",
    "CZ": "Czechia",
    "SK": "Slovakia",
    "HU": "Hungary",
    "RO": "Romania",
    "BG": "Bulgaria",
    "UA": "Ukraine",
}

# Date range you chose: 2018–2024
DATE_FROM = "2018-01-01"
DATE_TO   = "2024-12-31"


In [108]:
def download_country_pm25(iso_code, date_from, date_to, max_sensors=None):
    """
    Download daily PM2.5 data for all sensors (or a limited subset) in one country.
    
    iso_code: 'LV', 'EE', etc.
    date_from/date_to: strings 'YYYY-MM-DD'
    max_sensors: if not None, limits how many sensors to use (for testing).
    """
    print(f"\n=== {iso_code}: preparing sensor list ===")
    
    # Filter PM2.5 sensors in this country
    pm25_country = locations_df[
        (locations_df["country_code"] == iso_code) &
        (locations_df["parameter"].str.lower() == "pm25")
    ].drop_duplicates(subset=["sensor_id"])
    
    if pm25_country.empty:
        print(f"No PM2.5 sensors found for {iso_code}. Skipping.")
        return None
    
    sensor_ids = pm25_country["sensor_id"].unique()
    
    if max_sensors is not None:
        sensor_ids = sensor_ids[:max_sensors]
        print(f"Using only first {len(sensor_ids)} sensors (max_sensors={max_sensors}).")
    else:
        print(f"Found {len(sensor_ids)} PM2.5 sensors.")
    
    all_days = []
    
    # Loop over sensors
    for sid in sensor_ids:
        print(f"  Sensor {sid}: fetching daily data...")
        page = 1
        
        while True:
            try:
                data = get_daily_measurements_for_sensor(
                    sid,
                    date_from,
                    date_to,
                    limit=1000,
                    page=page
                )
            except Exception as e:
                print(f"    Error for sensor {sid} on page {page}: {e}")
                break  # move to next sensor
            
            df = days_to_dataframe(data, sid)
            
            if df.empty:
                # No data on this page -> done for this sensor
                break
            
            all_days.append(df)
            
            # If less than 1000 rows came back, it's probably the last page
            if len(df) < 1000:
                break
            
            page += 1
    
    if not all_days:
        print(f"No daily data downloaded for {iso_code}.")
        return None
    
    # Combine all daily rows
    country_days = pd.concat(all_days, ignore_index=True)
    
    # Merge in country/city/station/coords
    meta_unique = pm25_country.drop_duplicates(subset=["sensor_id"])
    
    country_full = country_days.merge(
        meta_unique[
            ["sensor_id", "country_code", "locality",
             "location_name", "latitude", "longitude"]
        ],
        on="sensor_id",
        how="left"
    )
    
    print(f"Finished {iso_code}: {country_full.shape[0]} rows, {country_full.shape[1]} columns.")
    return country_full


In [109]:
test_lv = download_country_pm25("LV", DATE_FROM, DATE_TO, max_sensors=5)
test_lv.head(), test_lv.shape



=== LV: preparing sensor list ===
Using only first 5 sensors (max_sensors=5).
  Sensor 1606344: fetching daily data...
  Sensor 1606348: fetching daily data...
  Sensor 1606345: fetching daily data...
  Sensor 7774665: fetching daily data...
  Sensor 7773036: fetching daily data...
Finished LV: 3453 rows, 10 columns.


(   sensor_id parameter  units  date_local  avg_value country_code locality  \
 0    1606344      pm25  µg/m³  2022-02-10       6.35           LV     Riga   
 1    1606344      pm25  µg/m³  2022-02-13      14.30           LV     Riga   
 2    1606344      pm25  µg/m³  2022-02-14      17.40           LV     Riga   
 3    1606344      pm25  µg/m³  2022-02-15      24.70           LV     Riga   
 4    1606344      pm25  µg/m³  2022-02-16      25.70           LV     Riga   
 
               location_name   latitude  longitude  
 0  Riga Kronvalda boulevard  56.954847  24.104756  
 1  Riga Kronvalda boulevard  56.954847  24.104756  
 2  Riga Kronvalda boulevard  56.954847  24.104756  
 3  Riga Kronvalda boulevard  56.954847  24.104756  
 4  Riga Kronvalda boulevard  56.954847  24.104756  ,
 (3453, 10))

In [115]:
results_summary = []

for iso_code in COUNTRIES.keys():
    print(f"\n######## Starting country {iso_code} ########")
    country_df = download_country_pm25(iso_code, DATE_FROM, DATE_TO, max_sensors=None)
    
    if country_df is None:
        print(f"Skipping {iso_code} (no data).")
        continue
    
    # Save to CSV
    output_path = f"../data/raw/{iso_code}_pm25_2018_2024.csv"
    country_df.to_csv(output_path, index=False)
    print(f"Saved {iso_code} data to {output_path}")
    
    # Store summary info
    results_summary.append({
        "iso_code": iso_code,
        "rows": country_df.shape[0],
        "columns": country_df.shape[1],
        "file": output_path,
    })

results_summary



######## Starting country EE ########

=== EE: preparing sensor list ===
Found 9 PM2.5 sensors.
  Sensor 23435: fetching daily data...
  Sensor 23453: fetching daily data...
  Sensor 8613037: fetching daily data...
  Sensor 23448: fetching daily data...
  Sensor 35347: fetching daily data...
  Sensor 8613075: fetching daily data...
  Sensor 14152440: fetching daily data...
  Sensor 13502144: fetching daily data...
  Sensor 14651174: fetching daily data...
Finished EE: 3367 rows, 10 columns.
Saved EE data to ../data/raw/EE_pm25_2018_2024.csv

######## Starting country LV ########

=== LV: preparing sensor list ===
Found 9 PM2.5 sensors.
  Sensor 1606344: fetching daily data...
  Sensor 1606348: fetching daily data...
  Sensor 1606345: fetching daily data...
  Sensor 7774665: fetching daily data...
  Sensor 7773036: fetching daily data...


KeyboardInterrupt: 